In [ ]:
import geopandas as gpd
import joblib
from joblib import Parallel, delayed
import math
from more_itertools import split_after
import numpy as np
import numpy.ma as ma
import os
import pandas as pd
import shapely
from shapely.ops import substring
from pyproj import CRS as CRS
from pyproj import Geod
import rasterio as rio
import rioxarray as riox
from shapely.geometry import Point, LineString
from tqdm import tqdm
import holoviews as hv
hv.extension('bokeh')
from holoviews import dim, opts
import hvplot.pandas
from pyproj import Geod
geod = Geod('+a=1737400.0')
GCS_Moon_2000 = CRS.from_wkt('GEOGCS["Moon 2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400.0,0.0]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]')
MoonEqui = CRS.from_wkt('PROJCS["Moon_Equidistant_Cylindrical",GEOGCS["Moon 2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400.0,0.0]],PRIMEM["Greenwich",0],UNIT["Decimal_Degree",0.0174532925199433]],PROJECTION["Equidistant_Cylindrical"],PARAMETER["False_Easting",0],PARAMETER["False_Northing",0],PARAMETER["Central_Meridian",0],PARAMETER["Standard_Parallel_1",0],UNIT["Meter",1]]')
from utils.utils import get_img_aff,geodataframe_reproj,parallel_geodensifier, cut, FindMaxLength, dem_profiler, giveline, parallel_transectifier, parallel_tReshaper, transect_dataframe_creator, transect_reshaper, chunk_creator

## Define initial Faults shapefile

In [ ]:
crater_name= 'Humboldt'

In [ ]:
faults_linear_shapes = f'./{crater_name}_Dec2022/faults-humboldt_v2.shp'

## Define basemaps files

In [ ]:
dem_file = f'./{crater_name}_Dec2022/{crater_name}-DEM.tif'
slope_file = f'./{crater_name}_Dec2022/{crater_name}-slope.tif'
aspect_file = f'./{crater_name}_Dec2022/{crater_name}-aspect.tif'
image = f'./{crater_name}_Dec2022/{crater_name}-WAC-ortho.tif'
basecrs = rio.open(image).crs
processing_folder = f'{os.path.dirname(image)}/processing'
os.makedirs(processing_folder,exist_ok=True)

## Read basemaps files

In [ ]:
dem_new_file, dem_aff, dem_img = get_img_aff(dem_file, basecrs)
slope_new_file, slope_aff, slope_img = get_img_aff(slope_file, basecrs)
aspect_new_file, asp_aff, aspect_img = get_img_aff(aspect_file, basecrs)
riox_dem = riox.open_rasterio(dem_file, masked=True)
dem_res = math.ceil(dem_aff[0])

## Reproject fault shapefile and densify each fault geometry
**Each fault geometry is composed by a number of points that corresponds to the user points created during the fault drawing**

**Densifying the fault geometry means we create equally spaced points along all the geometry**

In [ ]:
linear_reproj_gdf_file,linear_repr_gdf = geodataframe_reproj(processing_folder, faults_linear_shapes, basecrs)
linear_densified_gdf_file, linear_densified_gdf, chunk_results=parallel_geodensifier(processing_folder, linear_reproj_gdf_file, 'fault_id', 4, dem_res)
geographic_linear_densified_gdf=linear_densified_gdf.to_crs(GCS_Moon_2000)

## Aperture Statistics

In [ ]:
faults_copy = linear_densified_gdf.copy()

## Filter faults
**fault_drop_list contains all the id of anomalous graben faults (e.g. grabens with different faults length, graben with 3+ faults)**

In [ ]:
fault_drop_list = [3,4,5,6,7,8,13,14,27,28,29,30,47,49,50,51,115,116,117]
faults_drop_index = [faults_copy[faults_copy['fault_id']==flts].index.values[0] for flts in fault_drop_list]

## Fault pairs
**Generate graben fault pairs**

In [ ]:
indexes_to_keep = set(range(faults_copy.shape[0])) - set(faults_drop_index)
faults_copy_sliced_in = faults_copy.take(list(indexes_to_keep)).reset_index()
faults_copy_sliced_in.to_file (f'./{crater_name}_Dec2022/processing/filtered_simple.gpkg', driver='GPKG')

## Processing Fault Pairs

In [ ]:
complete_indxs = list(faults_copy_sliced_in.index)
indxs = [complete_indxs[i:i+2] for i in range(0, len(complete_indxs)-1, 2)]
start_indx = [ix[0] for ix in indxs]
stop_indx = [ix[1] for ix in indxs]

## Compute Graben Aperture
**To compute graben aperture, for each fault we measure the min, max and mean distance between the fault and its paired fault's points.**

In [ ]:
for ii, jj in indxs:
    #ii = 2
    #jj = 3
    series1=faults_copy_sliced_in.iloc[ii]
    series2=faults_copy_sliced_in.iloc[jj]
    id1=series1['fault_id']
    id2=series2['fault_id']
    l1 = series1.geometry
    l2 = series2.geometry
    l1_points = [Point(l1.xy[0][i],l1.xy[1][i]) for i in range(len(l1.xy[0]))]
    l2_points = [Point(l2.xy[0][i],l2.xy[1][i]) for i in range(len(l2.xy[0]))]
    l1_point_distance = []
    l2_point_distance = []
    for point in l1_points:
        pt_distance = point.distance(l2)
        l1_point_distance.append(pt_distance)            
    for point in l2_points:
        pt_distance = point.distance(l1)
        l2_point_distance.append(pt_distance)
    l1_percentile = np.percentile(l1_point_distance,75)
    l1_percentilemid = np.percentile(l1_point_distance,25)
    l2_percentile = np.percentile(l2_point_distance, 80)
    l2_percentilemid = np.percentile(l2_point_distance,25)
   #l1_distance = [dst if dst <l1_percentile else l1_percentilemid for dst in l1_point_distance]
   # l2_distance = [dst if dst <l2_percentile else l2_percentilemid for dst in l2_point_distance]
    #l1_distance = [dst if dst >=np.mean(l1_point_distance)//2 else np.mean(l2_point_distance) for dst in l1_point_distance]
    #l2_distance = [dst if dst >=np.mean(l2_point_distance)//2 else np.mean(l1_point_distance) for dst in l1_point_distance]
    #l1_distance = [dst if dst <l1_percentile else l1_percentilemid for dst in l1_distance]
    #l2_distance = [dst if dst <l2_percentile else l1_percentilemid for dst in l2_distance]
    #l1_distance = [dst if dst <=l1_percentile else l1_percentilemid for dst in l1_distance]
    #l2_distance = [dst if dst <=l2_percentile else l1_percentilemid for dst in l1_distance]
    l1_distance = [dst for dst in l1_point_distance]
    l2_distance = [dst for dst in l2_point_distance]
    zipped = list(zip(l1_point_distance,l2_point_distance))
    l1_aperture=[]
    l2_aperture=[]
    for z in zipped:
        ratio1 = z[0]/z[1]
        ratio2 = z[1]/z[0]
        if math.isclose(ratio1, 1,rel_tol=0.5):
            l1_aperture.append(z[0])
            l2_aperture.append(z[1])
        elif ratio1 > ratio2:
            l1_aperture.append(np.mean(l2_distance))
            l2_aperture.append(np.mean(l2_distance))
        else:
            l1_aperture.append(np.mean(l1_distance))
            l2_aperture.append(np.mean(l1_distance))
    l1_distance = l1_aperture
    l2_distance=l2_aperture
    faults_copy_sliced_in.loc[faults_copy_sliced_in['fault_id']==id1,'MinWidth(m)'] = np.min(l1_distance)
    faults_copy_sliced_in.loc[faults_copy_sliced_in['fault_id']==id1,'MaxWidth(m)'] = np.max(l1_distance)
    faults_copy_sliced_in.loc[faults_copy_sliced_in['fault_id']==id1,'MeanWidth(m)'] = np.mean(l1_distance)
    faults_copy_sliced_in.loc[faults_copy_sliced_in['fault_id']==id1,'StdErW(m)'] = np.std(l1_distance)
    faults_copy_sliced_in.loc[faults_copy_sliced_in['fault_id']==id1,'GrbMeW(m)'] = np.mean(l1_distance+l2_distance)
    faults_copy_sliced_in.loc[faults_copy_sliced_in['fault_id']==id2,'MinWidth(m)'] = np.min(l2_distance)
    faults_copy_sliced_in.loc[faults_copy_sliced_in['fault_id']==id2,'MaxWidth(m)'] = np.max(l2_distance)
    faults_copy_sliced_in.loc[faults_copy_sliced_in['fault_id']==id2,'MeanWidth(m)'] = np.mean(l2_distance)
    faults_copy_sliced_in.loc[faults_copy_sliced_in['fault_id']==id2,'StdErW(m)'] = np.std(l2_distance)
    faults_copy_sliced_in.loc[faults_copy_sliced_in['fault_id']==id2,'GrbMeW(m)'] = np.mean(l1_distance+l2_distance)
    #if ii == 2:
    #    break
#faults_copy_sliced_in['MeanWidth(m)']

## Transects generation
**Transects are generated at each point of each fault using the MeanWidth as length**

In [ ]:
from shapely.geometry import Point, LineString
geoLengths, segments, temp_datas = parallel_transectifier(faults_copy_sliced_in, 'MeanWidth(m)')

## Transect Dataframe creation

In [ ]:
transect_gdf_in = transect_dataframe_creator(temp_datas)

In [ ]:
transect_gdf_in.crs=basecrs
transect_gpkg_in = f'./{crater_name}_Dec2022/processing/transects-{crater_name}_simple.gpkg'
transect_gdf_in.to_file(transect_gpkg_in,driver='GPKG')


In [ ]:
transect_gdf_trim_in = transect_gdf_in.copy()

## Transects trimming
**For all the fault, each fault's transect is truncated at the intersectio of the corresponding paired fault**

In [ ]:
for ii, jj in indxs:       
    series1=faults_copy_sliced_in.iloc[ii]
    series2=faults_copy_sliced_in.iloc[jj]
    id1=series1['fault_id']
    id2=series2['fault_id']
    tmp1 = transect_gdf_trim_in[transect_gdf_trim_in['Fault ID']==id1]
    tmp2 = transect_gdf_trim_in[transect_gdf_trim_in['Fault ID']==id2]
    l1 = series1.geometry
    l2 = series2.geometry
    nnf1 = faults_copy_sliced_in[faults_copy_sliced_in['fault_id']==id1].iloc[0].geometry
    nnf2 = faults_copy_sliced_in[faults_copy_sliced_in['fault_id']==id2].iloc[0].geometry
    cutplots=[]
    for i in tmp1.index:        
        nnt1 = tmp1.loc[i,'geometry']
        nncut1 = nnt1.difference(nnf2)
        try:
            nncut1geom = nncut1.geoms[0]
        except Exception as e:
            #print(e)
            nncut1geom = nncut1
            pass
        transect_gdf_trim_in.loc[i,'geometry']=nncut1geom
    for i in tmp2.index:
        nnt2 = tmp2.loc[i,'geometry']
        nncut2 = nnt2.difference(nnf1)
        try:
            nncut2geom = nncut2.geoms[0]
        except:
            nncut2geom = nncut2
            pass
        transect_gdf_trim_in.loc[i,'geometry']=nncut2geom

    

In [ ]:
transect_gdf_trim_in.crs=basecrs
transect_gpkg_in = f'./{crater_name}_Dec2022/processing/transects-{crater_name}_simple_trimmed.gpkg'
transect_gdf_trim_in.to_file(transect_gpkg_in,driver='GPKG')


## Transect reshaping using DEM profiles
**For each transect, a DEM profile will be generated and then the minimum elevation point will be used to cut the transect**

In [ ]:
Displacements, Geoms = parallel_tReshaper(transect_gdf_trim_in,transect_reshaper,riox_dem,dem_res)

In [ ]:
transect_gdf_reshaped_in = transect_gdf_trim_in.copy()

In [ ]:
transect_gdf_reshaped_in['TrDsp (m)'] = Displacements
transect_gdf_reshaped_in['geometry']=Geoms

## Transect Displacement statistics


In [ ]:
for jj in transect_gdf_reshaped_in['Fault ID']:
    mask = transect_gdf_reshaped_in['Fault ID']==jj
    stder= transect_gdf_reshaped_in[mask]['TrDsp (m)'].std()
    mean= transect_gdf_reshaped_in[mask]['TrDsp (m)'].mean()
    transect_gdf_reshaped_in.loc[mask,'StdErDsp']=stder    

In [ ]:
transect_gdf_reshaped_in.crs=basecrs
transect_gpkg_in = f'./{crater_name}_Dec2022/processing/transects-{crater_name}_simple_reshaped.gpkg'
transect_gdf_reshaped_in.to_file(transect_gpkg_in,driver='GPKG')


## Summary Faults statistics 

In [ ]:
faults_stat_cols = ['Fault ID','Fault L(m)',
                    'MaxDsp(m)','MeanDsp(m)','StdErDsp',
                    'MinWidth(m)','MaxWidth(m)','MeanWidth(m)','StdErW(m)','geometry']
faults_gdf_in = faults_copy_sliced_in.copy()#gpd.GeoDataFrame(columns=faults_stat_cols)#, dtype='int64')
faults_gdf_in['Fault L(m)']=list(transect_gdf_reshaped_in.groupby('Fault ID')['Fault L(m)'].max())
faults_gdf_in['MaxDsp(m)']=list(transect_gdf_reshaped_in.groupby('Fault ID')['TrDsp (m)'].max())
faults_gdf_in['MeanDsp(m)']=list(transect_gdf_reshaped_in.groupby('Fault ID')['TrDsp (m)'].mean())
faults_gdf_in['StdErDsp']=list(transect_gdf_reshaped_in.groupby('Fault ID')['StdErDsp'].max())
faults_gdf_in['MinWidth(m)']=faults_copy_sliced_in['MinWidth(m)']
faults_gdf_in['MaxWidth(m)']=faults_copy_sliced_in['MaxWidth(m)']
faults_gdf_in['MeanWidth(m)']=faults_copy_sliced_in['MeanWidth(m)']
faults_gdf_in['StdErW(m)']=faults_copy_sliced_in['StdErW(m)']
faults_gdf_in['geometry']=faults_copy_sliced_in['geometry']
faults_gdf_in['Fault ID']=faults_copy_sliced_in['fault_id']

In [ ]:
faults_gdf_in.to_file (f'./{crater_name}_Dec2022/processing/filtered_simple.gpkg', driver='GPKG')


#Hang the execution to let the user perform manual operations.
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

raise StopExecution

## Complex Faults
**fault_drop_list contains all the id of anomalous graben faults (e.g. grabens with different faults length, graben with 3+ faults)**

In [ ]:
fault_groups_id = [[2,3],[3,4],[4,5],[5,6],[6,7],[12,13],[26,27],[26,28],[28,29],[47,49],[47,50],[46,47],[114,115],[115,116]]

In [ ]:
indexes_to_throw = set(range(faults_copy.shape[0])) - set(indexes_to_keep)
faults_copy_sliced_out = faults_copy.take(list(indexes_to_throw)).reset_index()
faults_copy_sliced_out.to_file (f'./{crater_name}_Dec2022/processing/filtered_complex.gpkg', driver='GPKG')

In [ ]:
complete_indxs = list(faults_copy_sliced_out.index)
indxs = [complete_indxs[i:i+2] for i in range(0, len(complete_indxs)-1, 2)]
start_indx = [ix[0] for ix in indxs]
stop_indx = [ix[1] for ix in indxs]

In [ ]:
uni = []
for u,q in fault_groups_id:
    uni.append(u)
    uni.append(q)
mapped = dict(zip(list(set(uni)),complete_indxs))

## Compute Graben Aperture
**To compute graben aperture, for each fault we measure the min, max and mean distance between the fault and its paired fault's points.**

In [ ]:
for ii, jj in fault_groups_id:
    series1=faults_copy_sliced_out.iloc[mapped[ii]]
    series2=faults_copy_sliced_out.iloc[mapped[jj]]
    id1=series1['fault_id']
    id2=series2['fault_id']
    l1 = series1.geometry
    l2 = series2.geometry
    l1_points = [Point(l1.xy[0][i],l1.xy[1][i]) for i in range(len(l1.xy[0]))]
    l2_points = [Point(l2.xy[0][i],l2.xy[1][i]) for i in range(len(l2.xy[0]))]
    l1_point_distance = []
    l2_point_distance = []
    for point in l1_points:
        pt_distance = point.distance(l2)
        l1_point_distance.append(pt_distance)            
    for point in l2_points:
        l2_point_distance.append(pt_distance)
        
    percentile = np.percentile(faults_gdf_in['MeanWidth(m)'], 97)
    percentilemid = np.percentile(faults_gdf_in['MeanWidth(m)'], 50)
    l1_distance = [dst if dst <=percentile else percentilemid*percentile/dst+percentilemid for dst in l1_point_distance]
    l2_distance = [dst if dst <=percentile else percentilemid*percentile/dst+percentilemid for dst in l2_point_distance]
    #l1_percentile = np.percentile(l1_point_distance,80)
    #l1_percentilemid = np.percentile(l1_point_distance,50)
    #l2_percentile = np.percentile(l2_point_distance, 80)
    #l2_percentilemid = np.percentile(l2_point_distance,50)
    #l1_distance = [dst if dst <l1_percentile else l1_percentilemid for dst in l1_point_distance]
    #l2_distance = [dst if dst <l2_percentile else l1_percentilemid for dst in l2_point_distance]
    
    
    
    zipped = list(zip(l1_distance,l2_distance))
    l1_aperture=[]
    l2_aperture=[]
    for z in zipped:
        ratio1 = z[0]/z[1]
        ratio2 = z[1]/z[0]
        if math.isclose(ratio1, 1,rel_tol=0.5):
            l1_aperture.append(z[0])
            l2_aperture.append(z[1])
        elif ratio1 > ratio2:
            l1_aperture.append(np.mean(l2_distance))
            l2_aperture.append(np.mean(l2_distance))
        else:
            l1_aperture.append(np.mean(l1_distance))
            l2_aperture.append(np.mean(l1_distance))
    l1_distance = l1_aperture
    l2_distance=l2_aperture
    faults_copy_sliced_out.loc[faults_copy_sliced_out['fault_id']==id1,'MinWidth(m)'] = np.min(l1_distance)
    faults_copy_sliced_out.loc[faults_copy_sliced_out['fault_id']==id1,'MaxWidth(m)'] = np.max(l1_distance)
    faults_copy_sliced_out.loc[faults_copy_sliced_out['fault_id']==id1,'MeanWidth(m)'] = np.mean(l1_distance)
    faults_copy_sliced_out.loc[faults_copy_sliced_out['fault_id']==id1,'StdErW(m)'] = np.std(l1_distance)
    faults_copy_sliced_out.loc[faults_copy_sliced_out['fault_id']==id2,'MinWidth(m)'] = np.min(l2_distance)
    faults_copy_sliced_out.loc[faults_copy_sliced_out['fault_id']==id2,'MaxWidth(m)'] = np.max(l2_distance)
    faults_copy_sliced_out.loc[faults_copy_sliced_out['fault_id']==id2,'MeanWidth(m)'] = np.mean(l2_distance)
    faults_copy_sliced_out.loc[faults_copy_sliced_out['fault_id']==id2,'StdErW(m)'] = np.std(l2_distance)

In [ ]:
from shapely.geometry import Point, LineString
geoLengths, segments, temp_datas = parallel_transectifier(faults_copy_sliced_out, 'MeanWidth(m)')

In [ ]:
transect_gdf_out = transect_dataframe_creator(temp_datas)

In [ ]:
transect_gdf_out.crs=basecrs
transect_gpkg_out = f'./{crater_name}_Dec2022/processing/transects-{crater_name}_complex.gpkg'
transect_gdf_out.to_file(transect_gpkg_out,driver='GPKG')

## Transects trimming
**For all the fault, each fault's transect is truncated at the intersectio of the corresponding paired fault**

In [ ]:
transect_gdf_trim_out = transect_gdf_out.copy()

In [ ]:
for ii, jj in fault_groups_id:

    series1=faults_copy_sliced_out.iloc[mapped[ii]]
    series2=faults_copy_sliced_out.iloc[mapped[jj]]
    id1=series1['fault_id']
    id2=series2['fault_id']
    tmp1 = transect_gdf_trim_out[transect_gdf_trim_out['Fault ID']==id1]
    tmp2 = transect_gdf_trim_out[transect_gdf_trim_out['Fault ID']==id2]
    l1 = series1.geometry
    l2 = series2.geometry
    nnf1 = faults_copy_sliced_out[faults_copy_sliced_out['fault_id']==id1].iloc[0].geometry
    nnf2 = faults_copy_sliced_out[faults_copy_sliced_out['fault_id']==id2].iloc[0].geometry
    cutplots=[]
    for i in tmp1.index:        
        nnt1 = tmp1.loc[i,'geometry']
        nncut1 = nnt1.difference(nnf2)
        try:
            nncut1geom = nncut1.geoms[0]
        except Exception as e:
            #print(e)
            nncut1geom = nncut1
            pass
        if nncut1geom.length >=1000:
            nncut1geom= cut(nncut1geom,1000)[0]
        transect_gdf_trim_out.loc[i,'geometry']=nncut1geom
    for i in tmp2.index:
        nnt2 = tmp2.loc[i,'geometry']
        nncut2 = nnt2.difference(nnf1)
        try:
            nncut2geom = nncut2.geoms[0]
        except:
            nncut2geom = nncut2
            pass
        if nncut2geom.length >=1000:
            nncut2geom= cut(nncut2geom,1000)[0]
        transect_gdf_trim_out.loc[i,'geometry']=nncut2geom

In [ ]:
transect_gdf_trim_out.crs=basecrs
transect_gdf_gpkg_out = f'./{crater_name}_Dec2022/processing/transects-{crater_name}_complex_trimmed.gpkg'
transect_gdf_trim_out.to_file(transect_gpkg_out,driver='GPKG')


## Transect reshaping using DEM profiles
**For each transect, a DEM profile will be generated and then the minimum elevation point will be used to cut the transect**

In [ ]:
Displacements, Geoms = parallel_tReshaper(transect_gdf_trim_out,transect_reshaper,riox_dem,dem_res)

In [ ]:
transect_gdf_reshaped_out = transect_gdf_trim_out.copy()

In [ ]:
transect_gdf_reshaped_out['TrDsp (m)'] = Displacements
transect_gdf_reshaped_out['geometry']=Geoms

## Summary Faults statistics 

In [ ]:
for jj in transect_gdf_reshaped_out['Fault ID']:
    mask = transect_gdf_reshaped_out['Fault ID']==jj
    stder= transect_gdf_reshaped_out[mask]['TrDsp (m)'].std()
    mean= transect_gdf_reshaped_out[mask]['TrDsp (m)'].mean()
    transect_gdf_reshaped_out.loc[mask,'StdErDsp']=stder    

In [ ]:
transect_gdf_reshaped_out.crs=basecrs
transect_gpkg_out = f'./{crater_name}_Dec2022/processing/transects-{crater_name}_complex_reshaped.gpkg'
transect_gdf_reshaped_out.to_file(transect_gpkg_out,driver='GPKG')


In [ ]:
faults_stat_cols = ['Fault ID','Fault L(m)',
                    'MaxDsp(m)','MeanDsp(m)','StdErDsp',
                    'MinWidth(m)','MaxWidth(m)','MeanWidth(m)','StdErW(m)','geometry']
faults_gdf_out = faults_copy_sliced_out.copy()#gpd.GeoDataFrame(columns=faults_stat_cols)#, dtype='int64')
faults_gdf_out['Fault L(m)']=list(transect_gdf_reshaped_out.groupby('Fault ID')['Fault L(m)'].max())
faults_gdf_out['MaxDsp(m)']=list(transect_gdf_reshaped_out.groupby('Fault ID')['TrDsp (m)'].max())
faults_gdf_out['MeanDsp(m)']=list(transect_gdf_reshaped_out.groupby('Fault ID')['TrDsp (m)'].mean())
faults_gdf_out['StdErDsp']=list(transect_gdf_reshaped_out.groupby('Fault ID')['StdErDsp'].max())
faults_gdf_out['MinWidth(m)']=faults_copy_sliced_out['MinWidth(m)']
faults_gdf_out['MaxWidth(m)']=faults_copy_sliced_out['MaxWidth(m)']
faults_gdf_out['MeanWidth(m)']=faults_copy_sliced_out['MeanWidth(m)']
faults_gdf_out['StdErW(m)']=faults_copy_sliced_out['StdErW(m)']
faults_gdf_out['geometry']=faults_copy_sliced_out['geometry']
faults_gdf_out['Fault ID']=faults_copy_sliced_out['fault_id']

In [ ]:
faults_gdf_out.to_file (f'./{crater_name}_Dec2022/processing/filtered_complex.gpkg', driver='GPKG')

## Combining data

In [ ]:
transect_unified = pd.concat([transect_gdf_reshaped_in, transect_gdf_reshaped_out]).reset_index()
transect_gdf_reshaped_in_f1 = transect_unified[transect_unified['Fault ID']!=117]
transect_gdf_reshaped_in_f1.to_file(f'./{crater_name}_Dec2022/processing/{crater_name}_transects_final.gpkg', driver='GPKG')

In [ ]:
faults_unified = pd.concat([faults_gdf_in,faults_gdf_out]).reset_index()
faults_gdf_in_f1 = faults_unified[faults_unified['Fault ID']!=117]
faults_gdf_in_f1.to_file(f'./{crater_name}_Dec2022/processing/{crater_name}_faults_final.gpkg', driver='GPKG')
faults_gdf_in_f1.to_excel(f'./{crater_name}_Dec2022/processing/{crater_name}_faults_final.xlsx')